In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
subset = 100
resave = False
window = 10

In [3]:
if resave:
    train = pd.read_csv('./raw_data/train.csv')
    train = train.head(subset)
    train_card_ids = train['card_id']

    hist_trans = []
    for chunk in pd.read_csv('./raw_data/historical_transactions.csv',chunksize=100000):
        hist_trans.append(chunk[chunk['card_id'].isin(train_card_ids)])

    new_merch_trans = []
    for chunk in pd.read_csv('./raw_data/new_merchant_transactions.csv',chunksize=100000):
        new_merch_trans.append(chunk[chunk['card_id'].isin(train_card_ids)])

    hist_trans = pd.concat(hist_trans)
    new_merch_trans = pd.concat(new_merch_trans)

    train.to_pickle('./raw_data/train_subset.pkl')
    hist_trans.to_pickle('./raw_data/hist_trans_subset.pkl')
    new_merch_trans.to_pickle('./raw_data/new_merch_subset.pkl')

In [4]:
train = pd.read_pickle('./raw_data/train_subset.pkl')
hist_trans = pd.read_pickle('./raw_data/hist_trans_subset.pkl')
new_merch_trans = pd.read_pickle('./raw_data/new_merch_subset.pkl')

In [5]:
window_df = hist_trans.groupby('card_id').apply(lambda x: x.sort_values('purchase_date',ascending=False,axis=0).head(window))
window_df = window_df.reset_index(drop=True)
window_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_022b48f3ac,292,N,0,A,307,M_ID_10f895765a,0,-0.520263,2017-12-30 17:00:57,3.0,1,19
1,Y,C_ID_022b48f3ac,69,N,0,A,879,M_ID_00a6ca8a8a,0,-0.698072,2017-12-30 16:53:51,1.0,9,29
2,Y,C_ID_022b48f3ac,69,N,0,A,879,M_ID_00a6ca8a8a,0,-0.608904,2017-12-30 16:42:46,1.0,9,29
3,Y,C_ID_022b48f3ac,69,N,0,A,879,M_ID_00a6ca8a8a,0,-0.521991,2017-12-30 16:40:26,1.0,9,29
4,N,C_ID_022b48f3ac,69,N,0,A,879,M_ID_00a6ca8a8a,0,-0.328540,2017-12-30 16:38:10,1.0,9,29


In [6]:
COL_SUBSET = ['card_id','authorized_flag','category_1','purchase_amount','month_lag']

In [7]:
window_df = window_df[COL_SUBSET]

In [8]:
authorized_flag_dict = {'Y':1,
                        'N':0}
window_df['authorized_flag'] = window_df['authorized_flag'].map(authorized_flag_dict)
window_df.head()

,card_id,authorized_flag,category_1,purchase_amount,month_lag
0,C_ID_022b48f3ac,1,N,-0.520263,0
1,C_ID_022b48f3ac,1,N,-0.698072,0
2,C_ID_022b48f3ac,1,N,-0.608904,0
3,C_ID_022b48f3ac,1,N,-0.521991,0
4,C_ID_022b48f3ac,0,N,-0.328540,0


In [9]:
category_1_dict = {'Y':1,
                   'N':0}
window_df['category_1'] = window_df['category_1'].map(category_1_dict)
window_df.head()

,card_id,authorized_flag,category_1,purchase_amount,month_lag
0,C_ID_022b48f3ac,1,0,-0.520263,0
1,C_ID_022b48f3ac,1,0,-0.698072,0
2,C_ID_022b48f3ac,1,0,-0.608904,0
3,C_ID_022b48f3ac,1,0,-0.521991,0
4,C_ID_022b48f3ac,0,0,-0.328540,0


In [10]:
def reshape_df(df_grp,key,window=10):
    
    def reshape_series(series):
        np_series = series.values.reshape(-1)
        if len(np_series) < 10:
            pad_zeros = window - len(series)
            np_series = np.append(np_series,np.zeros(pad_zeros))
        window_dict = {}
        for i, s in enumerate(np_series):
            window_dict['{}_{}'.format(series.name,i)] = s
        return(window_dict)
    
    full_dict = {}
    for col in df_grp.columns:
        if col == key:
            next
        else:
            full_dict.update(reshape_series(df_grp[col]))
    return(pd.Series(full_dict))

In [11]:
window_df_flattened = window_df.groupby('card_id').apply(lambda x: reshape_df(x,key='card_id'))
window_df_flattened.head()

,authorized_flag_0,authorized_flag_1,authorized_flag_2,authorized_flag_3,authorized_flag_4,authorized_flag_5,authorized_flag_6,authorized_flag_7,authorized_flag_8,authorized_flag_9,...,purchase_amount_0,purchase_amount_1,purchase_amount_2,purchase_amount_3,purchase_amount_4,purchase_amount_5,purchase_amount_6,purchase_amount_7,purchase_amount_8,purchase_amount_9
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_022b48f3ac,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,-0.520263,-0.698072,-0.608904,-0.521991,-0.328540,-0.328540,-0.584621,-0.589129,-0.682294,-0.629551
C_ID_04dd9152c7,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.735037,-0.735037,-0.735037,-0.735037,-0.745405,0.000000,0.000000,0.000000,0.000000,0.000000
C_ID_0894217f2f,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,-0.353213,-0.353664,-0.368540,-0.470420,-0.611819,-0.416625,-0.611669,-0.611669,-0.509789,-0.226990
C_ID_092d98ce80,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,-0.735097,-0.739395,-0.734887,-0.743902,-0.739395,-0.709342,-0.739560,-0.738448,-0.743752,-0.736389
C_ID_09da8a2858,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.716855,-0.731881,-0.731881,-0.708290,-0.691325,-0.689807,-0.378909,-0.145847,-0.596643,-0.596643


In [12]:
window_df_flattened = window_df_flattened.reset_index()
feature_vectors = window_df_flattened.columns[1:]

In [14]:
X = window_df_flattened[feature_vectors].values.reshape(len(window_df_flattened),len(COL_SUBSET)-1,window)
X = np.swapaxes(X,1,2)
Y = window_df_flattened[['card_id']].merge(train[['card_id','target']],how='left',on='card_id')['target'].values

In [15]:
from keras import Sequential
from keras.layers import Dense,TimeDistributed, LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

In [20]:
model = Sequential()
model.add(LSTM(16, input_shape=(window, len(COL_SUBSET)-1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 16)                1344      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,361
Trainable params: 1,361
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(X,Y,epochs=100)

Epoch 1/100
100/100 [==============================] - 1s 10ms/step - loss: 24.0750
Epoch 2/100
100/100 [==============================] - 0s 312us/step - loss: 24.0137
Epoch 3/100
100/100 [==============================] - 0s 312us/step - loss: 23.9845
Epoch 4/100
100/100 [==============================] - 0s 312us/step - loss: 23.9646
Epoch 5/100
100/100 [==============================] - 0s 312us/step - loss: 23.9325
Epoch 6/100
100/100 [==============================] - 0s 312us/step - loss: 23.9032
Epoch 7/100
100/100 [==============================] - 0s 312us/step - loss: 23.8616